In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown as md
np.random.seed(42)

In [2]:
pd.options.display.max_rows = 999

In [3]:
raw_df = pd.read_stata("./ARM_Data/nes/nes5200_processed_voters_realideo.dta")

In [4]:
raw_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
year,41498,NaN,NaN,NaN,1977.68,14.7177,1948,1966,1978,1990,2002
resid,41498,NaN,NaN,NaN,1325.85,1055.72,1,511,1125,1806,6009
weight1,41498,NaN,NaN,NaN,1.0565,0.375776,0.2417,1,1,1,4
weight2,41498,NaN,NaN,NaN,1.04031,0.39871,0,1,1,1,4
weight3,41498,NaN,NaN,NaN,1.03988,0.399297,0,1,1,1,4
age,39532,83,35,951,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,41395,2,2. female,22866,NaN,NaN,NaN,NaN,NaN,NaN,NaN
race,41185,6,1. white,34563,NaN,NaN,NaN,NaN,NaN,NaN,NaN
educ1,41059,4,"2. high school (12 grades or fewer, incl",19021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
urban,38794,3,"3. rural, small towns, outlying and adja",14328,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
lim_df = raw_df[~raw_df.vote.isna()][['gender', 'race', 'educ1', 'partyid7', 'ideo_feel', 'region', 'vote']]
lim_df

,gender,race,educ1,partyid7,ideo_feel,region,vote
0,1. male,1. white,1. grade school of less (0-8 grades),NaN,NaN,NaN,"2. yes, voted"
1,2. female,1. white,"2. high school (12 grades or fewer, incl",NaN,NaN,NaN,"2. yes, voted"
2,2. female,1. white,"2. high school (12 grades or fewer, incl",NaN,NaN,NaN,"2. yes, voted"
3,2. female,1. white,"3. some college(13 grades or more,but no",NaN,NaN,NaN,"2. yes, voted"
4,1. male,1. white,"3. some college(13 grades or more,but no",NaN,NaN,NaN,"2. yes, voted"
...,...,...,...,...,...,...,...
41491,2. female,1. white,4. college or advanced degree (no cases,6. weak republican,49. neutral,"1. northeast (ct,me,ma,nh,nj,ny,pa,ri,vt","2. yes, voted"
41493,1. male,1. white,4. college or advanced degree (no cases,2. weak democrat,29,"4. west (ak,az,ca,co,hi,id,mt,nv,nm,or,u","2. yes, voted"
41494,2. female,2. black,"3. some college(13 grades or more,but no",1. strong democrat,NaN,"4. west (ak,az,ca,co,hi,id,mt,nv,nm,or,u","2. yes, voted"
41495,2. female,1. white,"2. high school (12 grades or fewer, incl",5. independent-republican,74,"2. north central(il,in,ia,ks,mi,mn,mo,ne","2. yes, voted"


In [6]:
lim_df.describe(include='all')

,gender,race,educ1,partyid7,ideo_feel,region,vote
count,37986,37806,37697,36817,25635,37555,38088
unique,2,6,4,7,91,4,2
top,2. female,1. white,"2. high school (12 grades or fewer, incl",2. weak democrat,49. neutral,"3. south (al,ar,de,d.c.,fl,ga,ky,la,md,m","2. yes, voted"
freq,20999,31840,17447,8083,7714,12349,27319


`ideo_feel` has too many unique values, some tranformation is needed

In [7]:
lim_df.groupby("ideo_feel").aggregate("count")

,gender,race,educ1,partyid7,region,vote
ideo_feel,,,,,,
00. most liberal,96,96,96,95,96,96
1.0,2,2,2,2,2,2
2.0,3,3,3,3,3,3
4.0,6,6,6,6,6,6
5.0,5,4,5,5,5,5
6.0,41,41,41,40,41,41
8.0,47,47,46,47,47,47
9.0,25,25,25,25,25,25
10.0,9,9,9,9,9,9


In [8]:
clean_df = lim_df.assign(
    ideo_c=lambda x: x.ideo_feel.str.split('.').str[0].fillna(49).astype('int'),
    ideo_q=lambda x: (x.ideo_c.mean() - x.ideo_c)/2*x.ideo_c.std(),
    voted=lambda x: x.vote.str.contains("yes").astype('int')
)

In [9]:
mdl_full = smf.logit(data = clean_df, formula="voted ~ gender + race + educ1 + partyid7 + region + ideo_q").fit()

Optimization terminated successfully.
         Current function value: 0.556879
         Iterations 6


In [10]:
mdl_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  voted   No. Observations:                36397
Model:                          Logit   Df Residuals:                    36377
Method:                           MLE   Df Model:                           19
Date:                Sun, 22 Mar 2020   Pseudo R-squ.:                 0.05345
Time:                        21:01:11   Log-Likelihood:                -20269.
converged:                       True   LL-Null:                       -21413.
Covariance Type:            nonrobust   LLR p-value:                     0.000
======================================================================================================================
                                                         coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              1.4592      0.049     30.034      0.000       1.364       1.554
gender[T.2. female]                                   -0.1104      0.025     -4.476      0.000      -0.159      -0.062
race[T.2. black]                                      -0.3571      0.040     -9.040      0.000      -0.435      -0.280
race[T.3. asian]                                      -0.5332      0.136     -3.910      0.000      -0.800      -0.266
race[T.4. native american]                            -0.4303      0.090     -4.763      0.000      -0.607      -0.253
race[T.5. hispanic]                                   -0.5274      0.067     -7.908      0.000      -0.658      -0.397
race[T.7. other]                                      -0.5862      0.208     -2.813      0.005      -0.995      -0.178
educ1[T.2. high school (12 grades or fewer, incl]      0.1683      0.035      4.865      0.000       0.101       0.236
educ1[T.3. some college(13 grades or more,but no]      0.5145      0.041     12.447      0.000       0.433       0.596
educ1[T.4. college or advanced degree (no cases]       0.9585      0.045     21.115      0.000       0.870       1.047
partyid7[T.2. weak democrat]                          -0.6107      0.038    -16.039      0.000      -0.685      -0.536
partyid7[T.3. independent-democrat]                   -0.7517      0.046    -16.376      0.000      -0.842      -0.662
partyid7[T.4. independent-independent]                -1.0223      0.046    -22.208      0.000      -1.113      -0.932
partyid7[T.5. independent-republican]                 -0.4984      0.050    -10.027      0.000      -0.596      -0.401
partyid7[T.6. weak republican]                        -0.4990      0.044    -11.375      0.000      -0.585      -0.413
partyid7[T.7. strong republican]                       0.1834      0.052      3.516      0.000       0.081       0.286
region[T.2. north central(il,in,ia,ks,mi,mn,mo,ne]    -0.0868      0.036     -2.395      0.017      -0.158      -0.016
region[T.3. south (al,ar,de,d.c.,fl,ga,ky,la,md,m]    -0.5782      0.035    -16.694      0.000      -0.646      -0.510
region[T.4. west (ak,az,ca,co,hi,id,mt,nv,nm,or,u]    -0.1179      0.041     -2.841      0.005      -0.199      -0.037
ideo_q                                                -0.0012      0.001     -1.186      0.235      -0.003       0.001
======================================================================================================================
"""

In [11]:
mdl_cut = smf.logit(data = clean_df, formula="voted ~ educ1 + race + gender + partyid7").fit()
mdl_cut.summary()

Optimization terminated successfully.
         Current function value: 0.562365
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  voted   No. Observations:                36397
Model:                          Logit   Df Residuals:                    36381
Method:                           MLE   Df Model:                           15
Date:                Sun, 22 Mar 2020   Pseudo R-squ.:                 0.04412
Time:                        21:01:11   Log-Likelihood:                -20468.
converged:                       True   LL-Null:                       -21413.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.1777      0.041     28.858      0.000       1.098       1.258
educ1[T.2. high school (12 grades or fewer, incl]     0.1963      0.034      5.721      0.000       0.129       0.264
educ1[T.3. some college(13 grades or more,but no]     0.5334      0.041     13.045      0.000       0.453       0.614
educ1[T.4. college or advanced degree (no cases]      0.9685      0.045     21.508      0.000       0.880       1.057
race[T.2. black]                                     -0.4827      0.039    -12.492      0.000      -0.558      -0.407
race[T.3. asian]                                     -0.5255      0.135     -3.881      0.000      -0.791      -0.260
race[T.4. native american]                           -0.5399      0.089     -6.036      0.000      -0.715      -0.365
race[T.5. hispanic]                                  -0.6064      0.066     -9.239      0.000      -0.735      -0.478
race[T.7. other]                                     -0.4650      0.208     -2.237      0.025      -0.872      -0.058
gender[T.2. female]                                  -0.1070      0.025     -4.364      0.000      -0.155      -0.059
partyid7[T.2. weak democrat]                         -0.5992      0.038    -15.840      0.000      -0.673      -0.525
partyid7[T.3. independent-democrat]                  -0.6906      0.045    -15.186      0.000      -0.780      -0.602
partyid7[T.4. independent-independent]               -0.9817      0.046    -21.515      0.000      -1.071      -0.892
partyid7[T.5. independent-republican]                -0.4683      0.049     -9.497      0.000      -0.565      -0.372
partyid7[T.6. weak republican]                       -0.4396      0.043    -10.108      0.000      -0.525      -0.354
partyid7[T.7. strong republican]                      0.2326      0.052      4.500      0.000       0.131       0.334
=====================================================================================================================
"""

In [12]:
mdl_int = smf.logit(data = clean_df, formula="voted ~ educ1 + race*gender + partyid7").fit()
mdl_int.summary()

Optimization terminated successfully.
         Current function value: 0.562247
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  voted   No. Observations:                36397
Model:                          Logit   Df Residuals:                    36376
Method:                           MLE   Df Model:                           20
Date:                Sun, 22 Mar 2020   Pseudo R-squ.:                 0.04432
Time:                        21:01:11   Log-Likelihood:                -20464.
converged:                       True   LL-Null:                       -21413.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.1914      0.041     28.799      0.000       1.110       1.272
educ1[T.2. high school (12 grades or fewer, incl]     0.1974      0.034      5.752      0.000       0.130       0.265
educ1[T.3. some college(13 grades or more,but no]     0.5343      0.041     13.062      0.000       0.454       0.614
educ1[T.4. college or advanced degree (no cases]      0.9692      0.045     21.510      0.000       0.881       1.058
race[T.2. black]                                     -0.5380      0.061     -8.874      0.000      -0.657      -0.419
race[T.3. asian]                                     -0.7618      0.188     -4.042      0.000      -1.131      -0.392
race[T.4. native american]                           -0.5068      0.140     -3.616      0.000      -0.782      -0.232
race[T.5. hispanic]                                  -0.7610      0.098     -7.733      0.000      -0.954      -0.568
race[T.7. other]                                     -0.5870      0.296     -1.985      0.047      -1.167      -0.007
gender[T.2. female]                                  -0.1311      0.027     -4.840      0.000      -0.184      -0.078
partyid7[T.2. weak democrat]                         -0.6002      0.038    -15.864      0.000      -0.674      -0.526
partyid7[T.3. independent-democrat]                  -0.6924      0.045    -15.221      0.000      -0.782      -0.603
partyid7[T.4. independent-independent]               -0.9834      0.046    -21.543      0.000      -1.073      -0.894
partyid7[T.5. independent-republican]                -0.4707      0.049     -9.542      0.000      -0.567      -0.374
partyid7[T.6. weak republican]                       -0.4402      0.044    -10.117      0.000      -0.525      -0.355
partyid7[T.7. strong republican]                      0.2334      0.052      4.513      0.000       0.132       0.335
race[T.2. black]:gender[T.2. female]                  0.0914      0.077      1.194      0.233      -0.059       0.241
race[T.3. asian]:gender[T.2. female]                  0.4658      0.270      1.728      0.084      -0.062       0.994
race[T.4. native american]:gender[T.2. female]       -0.0546      0.182     -0.300      0.764      -0.411       0.302
race[T.5. hispanic]:gender[T.2. female]               0.2752      0.132      2.088      0.037       0.017       0.533
race[T.7. other]:gender[T.2. female]                  0.2366      0.416      0.570      0.569      -0.578       1.051
=====================================================================================================================
"""

In [13]:
mdl_int_cut = smf.logit(data = clean_df, formula="voted ~ educ1 + race:gender + partyid7").fit()
mdl_int_cut.summary()

Optimization terminated successfully.
         Current function value: 0.562247
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  voted   No. Observations:                36397
Model:                          Logit   Df Residuals:                    36376
Method:                           MLE   Df Model:                           20
Date:                Sun, 22 Mar 2020   Pseudo R-squ.:                 0.04432
Time:                        21:01:11   Log-Likelihood:                -20464.
converged:                       True   LL-Null:                       -21413.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.1914      0.041     28.799      0.000       1.110       1.272
educ1[T.2. high school (12 grades or fewer, incl]     0.1974      0.034      5.752      0.000       0.130       0.265
educ1[T.3. some college(13 grades or more,but no]     0.5343      0.041     13.062      0.000       0.454       0.614
educ1[T.4. college or advanced degree (no cases]      0.9692      0.045     21.510      0.000       0.881       1.058
partyid7[T.2. weak democrat]                         -0.6002      0.038    -15.864      0.000      -0.674      -0.526
partyid7[T.3. independent-democrat]                  -0.6924      0.045    -15.221      0.000      -0.782      -0.603
partyid7[T.4. independent-independent]               -0.9834      0.046    -21.543      0.000      -1.073      -0.894
partyid7[T.5. independent-republican]                -0.4707      0.049     -9.542      0.000      -0.567      -0.374
partyid7[T.6. weak republican]                       -0.4402      0.044    -10.117      0.000      -0.525      -0.355
partyid7[T.7. strong republican]                      0.2334      0.052      4.513      0.000       0.132       0.335
gender[T.2. female]                                  -0.1311      0.027     -4.840      0.000      -0.184      -0.078
race[T.2. black]:gender[1. male]                     -0.5380      0.061     -8.874      0.000      -0.657      -0.419
race[T.3. asian]:gender[1. male]                     -0.7618      0.188     -4.042      0.000      -1.131      -0.392
race[T.4. native american]:gender[1. male]           -0.5068      0.140     -3.616      0.000      -0.782      -0.232
race[T.5. hispanic]:gender[1. male]                  -0.7610      0.098     -7.733      0.000      -0.954      -0.568
race[T.7. other]:gender[1. male]                     -0.5870      0.296     -1.985      0.047      -1.167      -0.007
race[T.2. black]:gender[2. female]                   -0.4467      0.049     -9.155      0.000      -0.542      -0.351
race[T.3. asian]:gender[2. female]                   -0.2960      0.194     -1.529      0.126      -0.675       0.083
race[T.4. native american]:gender[2. female]         -0.5615      0.116     -4.830      0.000      -0.789      -0.334
race[T.5. hispanic]:gender[2. female]                -0.4859      0.088     -5.534      0.000      -0.658      -0.314
race[T.7. other]:gender[2. female]                   -0.3504      0.292     -1.200      0.230      -0.923       0.222
=====================================================================================================================
"""

* So we've kicked out the ideology variable as it was shown to be insignificant.
* out of all interactions between race and gender only the case of `hispanic` and `female` combination was an important factor when separate field values were present. Removing the fields themselves and leaving only combinations shown that some of other combinations became significant, yet, it didn't improve the model performance.